In [ ]:
!pip install nnfs==0.5.1

In [ ]:
import numpy as np


np.random.seed(0)

class Layer_Dense:
    def __init__ (self, n_inputs, n_neurons):
        self.weights = np.random.randn(n_inputs, n_neurons) * 0.01
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)

        self.dinputs = np.dot(dvalues, self.weights.T)

class Activation_ReLU:
    def __init__ (self):
        pass

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0.0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs

        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)


        # single_output은 한개의 배치에 대한, 각 인풋에 따른 softMax값
        # single_dvalues는 각 인풋에 따른 손실함수의 값

        # 각 배치에서 matrix 형태가 나오니, 배치를 합산하면 3D 이 문제를 다시 2D로 합산하기 위해, jacobian_matrix를 각 gradient로 내적을 시켜 합산함.
        for index, (single_output, single_dvalues) in \
                enumerate(zip(self.output, dvalues)):

            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)

            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

class Loss:
    def calculate (self, output, y):
        sample_losses = self.forward(output, y)
        # print('sample_losses : ', sample_losses[:5])
        data_loss = np.mean(sample_losses)
        return data_loss

    def backward (self, dvalues, y_true):
        samples = len(dvalues)

        # 분류해야할 클래스의 갯수, 마지막에 나오는 features 의 갯수.
        labels = len(dvalues[0])

        if len(y_true) == 1:
            y_true = np.eye(labels)[y_true]

        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples




class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            # y_true is Classifified Vector
            correct_confidences = y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape) == 2:
            # y_true is OneHotEncoded Vector
            correct_confidences = np.sum(y_pred_clipped * y_true, axis = 1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output

        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):

        samples = len(dvalues)


        # onehotencoding 된 y_true를 다시, 인덱스화 시킴.
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)

        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1

        self.dinputs = self.dinputs / samples;




In [ ]:
class Optimizer_SGD:
    def __init__(self, learning_rate = 1.0, decay = 0., momentum = 0.):
        self.learning_rate = learning_rate
        self.current_larning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def update_params(self, layer):

        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            weight_updates = \
                self.momentum * layer.weight_momentums - \
                self.current_larning_rate * layer.dweights

            layer.weight_momentums = weight_updates

            bias_updates = \
                self.momentum * layer.bias_momentums -\
                self.current_larning_rate * layer.dbiases

            layer.bias_momentums = bias_updates

        else :
            weight_updates = -self.current_larning_rate  * layer.dweights
            bias_updates = -self.current_larning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    def pre_update_params(self):
        if self.decay:
            self.current_larning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    def post_update_params(self):
        self.iterations += 1


class Optimizer_Adagrad:
    def __init__(self, learning_rate = 1.0, decay = 0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_larning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    def update_params(self, layer):

        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_cache += layer.dweights ** 2
        layer.bias_cache += layer.dbiases ** 2

        layer.weights += -self.current_larning_rate * layer.dweights \
                    / (np.sqrt(layer.weight_cache) + self.epsilon)

        layer.biases += -self.current_larning_rate * layer.dbiases \
                    / (np.sqrt(layer.bias_cache) + self.epsilon)

    def pre_update_params(self):
        if self.decay:
            self.current_larning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    def post_update_params(self):
        self.iterations += 1


class Optimizer_RMSprop:


    # RMSprop에서 학습률 감쇠가 느리기 때문에, learning rate 1 은 불안정성을 초래함.

    def __init__(self, learning_rate = 0.001, decay = 0., epsilon=1e-7, rho = 0.9):
        self.learning_rate = learning_rate
        self.current_larning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho

    def update_params(self, layer):

        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)


        # rho 가 클수록 과거의 값이 더 많이 반영됨. 일반적으로 0.9로 조정됨.
        layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights ** 2
        layer.bias_cache = self.rho * layer.bias_cache + (1 - self.rho) * layer.dbiases ** 2

        layer.weights += -self.current_larning_rate * layer.dweights \
                    / (np.sqrt(layer.weight_cache) + self.epsilon)

        layer.biases += -self.current_larning_rate * layer.dbiases \
                    / (np.sqrt(layer.bias_cache) + self.epsilon)

    def pre_update_params(self):
        if self.decay:
            self.current_larning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    def post_update_params(self):
        self.iterations += 1


class Optimizer_Adam:
    # RMSprop에서 학습률 감쇠가 느리기 때문에, learning rate 1 은 불안정성을 초래함.

    def __init__(self, learning_rate = 0.001, decay = 0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_larning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
            layer.bias_momentums = np.zeros_like(layer.biases)

        layer.weight_momentums = self.beta_1 * \
                                layer.weight_momentums + \
                                (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * \
                                layer.bias_momentums + \
                                (1 - self.beta_1) * layer.dbiases

        # 편향을 보정하는 것임. 초반에는 많이 곱하고, 후반에 그대로 사용하는것임.
        weight_momentums_corrected = layer.weight_momentums / \
                (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / \
                (1 - self.beta_1 ** (self.iterations + 1))

        # 여기는 RMSprops와 같음.
        layer.weight_cache = self.beta_2 * layer.weight_cache + \
                (1 - self.beta_2 ) * layer.dweights ** 2
        layer.bias_cache = self.beta_2 * layer.bias_cache + \
                (1 - self.beta_2 ) * layer.dbiases ** 2

        # 편향을 보정하는 것임. 초반에는 많이 곱하고, 후반에 그대로 사용하는것임.
        weight_cache_corrected = layer.weight_cache / \
            (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / \
            (1 - self.beta_2 ** (self.iterations + 1))

        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_larning_rate * \
                         weight_momentums_corrected / \
                         (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_larning_rate * \
                         bias_momentums_corrected / \
                         (np.sqrt(bias_cache_corrected) + self.epsilon)



    def pre_update_params(self):
        if self.decay:
            self.current_larning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    def post_update_params(self):
        self.iterations += 1

In [ ]:
import nnfs
nnfs.init()

from nnfs.datasets import spiral_data

X, y = spiral_data(100, 3)

dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(64, 3)
loss_function = Activation_Softmax_Loss_CategoricalCrossentropy()

# optimizer = Optimizer_SGD(learning_rate=1., decay=0.001, momentum=0.9)
# optimizer = Optimizer_Adagrad(learning_rate=1., decay=0.0001)
# optimizer = Optimizer_RMSprop(learning_rate=0.02, decay=1e-5, rho = 0.999)
optimizer = Optimizer_Adam(learning_rate=0.02, decay=1e-5, epsilon=1e-7, beta_1=0.9, beta_2=0.999)

for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)

    dense2.forward(activation1.output)
    loss = loss_function.forward(dense2.output, y)

    predictions = np.argmax(loss_function.output, axis= 1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)

    accuracy = np.mean(predictions == y)

    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, ' +
                f'Acc: {accuracy:.3f}, ' +
              f'Loss: {loss:.3f}, ' +
              f'learn_rate: {optimizer.current_larning_rate}')

    loss_function.backward(loss_function.output, y)
    dense2.backward(loss_function.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


Epoch: 0, Acc: 0.360, Loss: 1.099, learn_rate: 0.02
Epoch: 100, Acc: 0.673, Loss: 0.769, learn_rate: 0.01998021958261321
Epoch: 200, Acc: 0.813, Loss: 0.552, learn_rate: 0.019960279044701046
Epoch: 300, Acc: 0.840, Loss: 0.445, learn_rate: 0.019940378268975763
Epoch: 400, Acc: 0.863, Loss: 0.380, learn_rate: 0.01992051713662487
Epoch: 500, Acc: 0.873, Loss: 0.336, learn_rate: 0.01990069552930875
Epoch: 600, Acc: 0.890, Loss: 0.308, learn_rate: 0.019880913329158343
Epoch: 700, Acc: 0.890, Loss: 0.287, learn_rate: 0.019861170418772778
Epoch: 800, Acc: 0.907, Loss: 0.269, learn_rate: 0.019841466681217078
Epoch: 900, Acc: 0.910, Loss: 0.250, learn_rate: 0.01982180200001982
Epoch: 1000, Acc: 0.913, Loss: 0.238, learn_rate: 0.019802176259170884
Epoch: 1100, Acc: 0.910, Loss: 0.230, learn_rate: 0.01978258934311912
Epoch: 1200, Acc: 0.913, Loss: 0.219, learn_rate: 0.01976304113677013
Epoch: 1300, Acc: 0.920, Loss: 0.210, learn_rate: 0.019743531525483964
Epoch: 1400, Acc: 0.910, Loss: 0.202, le